In [ ]:

import matplotlib
matplotlib.use("PDF")
import pyfits as pf
from pylab import *
import sys, glob
from scipy import stats

thedir='/Users/cslage/Research/LSST/optical_simulator/CTE/'
%cd $thedir

#%matplotlib inline

In [ ]:
#files = glob.glob('ITL-3800C-029_flat_light_004_20161010??????.fits')
files = sort(glob.glob('ITL-3800C-029_flat_flat_008_20161102??????.fits'))
name = files[0][24:27]
print files
print name


In [ ]:
xstart = 505
xstop = 542
ov_start = 512
ystart = 200
ystop = 1800
xaxis = linspace(xstart,xstop-1,xstop-xstart)

for file in files:
    name = file[24:27]
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
    figure(figsize=(15,15))
    subplots_adjust(hspace = 0.5, wspace = 0.5)
    suptitle("Serial CTE by segment - ITL CCD 029 - %s"%name, fontsize = 16)

    for segment in range(16):
        try:
            subplot(4,4,segment+1)
            imhdr=hdulist[segment+1].header
            extname = imhdr['EXTNAME']
            title("CTE - %s"%extname)
            data = array(hdulist[segment+1].data + 32768, dtype = int32)
            flat_overscan = array(data[:,xstop-8:xstop].sum(axis = 1) / 8.0)
            cte_data = ((transpose(transpose(data) - flat_overscan))[ystart:ystop,:].mean(axis=0))[xstart:xstop]
            #print flat_overscan
            #print cte_data
            #sys.exit()
            cte_std = ((transpose(transpose(data) - flat_overscan))[ystart:ystop,:].std(axis=0) / sqrt(float(ystop-ystart)))[xstart:xstop]
            cti = median((transpose(transpose(data) - flat_overscan))[ystart:ystop,ov_start]\
            / (transpose(transpose(data) - flat_overscan))[ystart:ystop,ov_start-1]) / ov_start
            scatter(xaxis, cte_data, color = 'blue')
            errorbar(xaxis, cte_data, yerr=cte_std)
            text(xstart + 8, 20000, "CTI = %.2E"%cti, fontsize = 12)
            yscale('log',nonposy='clip')
            ylim(.01, 100000)
            xticks([510,520,530,540])
            xlabel("X (First overscan = 512)")
            ylabel("Signal(ADU)")
            print "Segment %s done"%extname
        except:
            continue
    savefig("Serial_CTE_%s_30Nov16.pdf"%name)
    clf()

In [ ]:
from scipy import stats
fig = figure()
fig.subplots_adjust(hspace = 0.5)
fig.suptitle("CTI Improvement with Delay - RG-up to IN1 Start")
#time = array([450, 650, 950, 1450, 2450, 4450, 8450])
time = array([300, 1100, 1300, 1600, 2100, 3100, 5100, 9100])
#time = array([0,1650, 1850, 2150, 2650, 3650, 5650, 8650])
CTI = array([11.2,5.97, 5.19, 3.45, 2.16, 1.79, 0.75, 0.61])
plotcounter=1
for CTI0 in [0.6]:
    ax1=fig.add_subplot(1,1,plotcounter)
    #title("CTI0 = %.2g"%(CTI0 * 1.0E-6))
    ax1.scatter(time, CTI)
    slope, intercept, r_value, p_value, std_err = stats.linregress(time,log(CTI-CTI0))
    xplot=linspace(0,10000,100)
    yplot = CTI0 + exp(intercept + slope * xplot)
    ax1.plot(xplot, yplot, ls = '--', lw = 2, color = 'red')
    ax1.plot([-200,10000],[5.0,5.0], ls = '--', lw = 1, color = 'green')
    ax1.text(5000, 5.5, "Spec")
    ax1.text(1000, 0.10, "CTI = CTI0 + A exp(-T/Tau)")
    ax1.plot([5200, 7000], [0.11,0.11], ls = '--', lw = 2, color = 'red')
    ax1.text(1000, 0.05, "Tau = %.0f nanoseconds"%(-1.0/slope))
    ax1.text(1000, 0.025, "A = %.2f E-6"%(exp(intercept)))
    ax1.text(1000, 0.0125, "CTI0= %.2g "%(CTI0 * 1.0E-6))
    ax1.set_yscale('log')
    ax1.set_ylim(0.01,100.0)
    ax1.yaxis.tick_left()
    ax1.set_xlim(-200, 10000)
    ax1.set_ylabel("CTI * 1.0E6")
    ax1.set_xlabel("Delay(nanoseconds)")
    ax2 = fig.add_subplot(111, sharex=ax1, frameon=False)
    ax2.yaxis.tick_right()
    ax2.yaxis.set_label_position("right")
    ax2.set_ylabel("CTI Ratio (Trailing/Peak)")
    ax2.set_yscale('log')
    ax2.set_ylim(5.12E-6,5.12E-2)

    plotcounter+=1
savefig("CTI_Delay_30Nov16.pdf")
    


In [ ]:
ax1 = fig1.add_subplot(111)
line1 = ax1.plot([1,3,4,5,2], 'o-')
ylabel("Left Y-Axis Data")
 
# now, the second axes that shares the x-axis with the ax1
ax2 = fig1.add_subplot(111, sharex=ax1, frameon=False)
line2 = ax2.plot([10,40,20,30,50], 'xr-')
ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")
ylabel("Right Y-Axis Data")